In [93]:
from abc import ABC, abstractmethod
from collections import defaultdict
import math
import networkx as nx
import numpy as np
import os
import random
from matplotlib import pyplot as plt

In [355]:
class MCTS:
    "Monte Carlo tree searcher. First rollout the tree then choose a move."

    def __init__(self, net, exploration_weight=1.5):
        self.Q = defaultdict(int)  # total reward of each node
        self.N = defaultdict(int)  # total visit count for each node
        self.children = dict()  # children of each node
        self.priors = dict() # prior probability of visiting each child of a given node
        self.exploration_weight = exploration_weight
        self.net = net

    def choose(self, node):
        "Choose the best successor of node. (Choose a move in the game)"
        if node.is_terminal():
            raise RuntimeError(f"choose called on terminal node {node}")

        if node not in self.children:
            return node.find_random_child()

        def score(n):
            if self.N[n] == 0:
                return float("-inf")  # avoid unseen moves
            return self.Q[n] / self.N[n]  # average reward

        return max(self.children[node], key=score)
    
    def choose_by_policy(self, node):
        "Choose a successor of node according to policy"
        if node.is_terminal():
            raise RuntimeError(f"choose called on terminal node {node}")
            
        if node not in self.children:
            return node.find_random_child()
        
        policy = list(map(lambda n: self.N[n]/(self.N[node]-1), self.children[node]))
        action = np.random.choice(len(policy), 1, p=policy)[0]
        successor = self.children[node][action]

        return successor,policy

    def do_rollout(self, node):
        "Make the tree one layer better. (Train for one iteration.)"
        path = self._select(node)
        leaf = path[-1]
        self._expand(leaf)
        reward = self._simulate(leaf)
        self._backpropagate(path, reward)

    def _select(self, node):
        "Find an unexplored descendent of `node`"
        path = []
        while True:
            path.append(node)
            if node not in self.children or not self.children[node]:
                # node is either unexplored or terminal
                return path
            for child in self.children[node]:
                if child not in self.children:
                    path.append(child)
                    return path
            node = self._uct_select(node)  # descend a layer deeper

    def _expand(self, node):
        "Update the `children` dict with the children of `node`"
        if node in self.children:
            return  # already expanded
        children = node.find_children()
        self.children[node] = children
        if children:
            self.priors[node] = list(map(lambda x: x*len(children), self.net.predict(node))

    def _simulate(self, node):
        "Returns the reward for a random simulation (to completion) of `node`"
        reward = 0
        while True:
            reward += node.get_reward()
            if node.is_terminal():
                return reward
            node = node.find_random_child()

    def _backpropagate(self, path, reward):
        "Send the reward back up to the ancestors of the leaf"
        for node in reversed(path):
            self.N[node] += 1
            self.Q[node] += reward

    def _uct_select(self, node):
        "Select a child of node, balancing exploration & exploitation"

        # All children of node should already be expanded:
        assert all(n in self.children for n in self.children[node])

        log_N_vertex = math.log(self.N[node])

        def uct(n):
            "Upper confidence bound for trees"
            i,n = n # expand from enumerate tuple
            return self.Q[n] / self.N[n] + self.exploration_weight * self.priors[node][i] * math.sqrt(
                log_N_vertex / self.N[n]
            )

        return max(enumerate(self.children[node]), key=uct)[1]


SyntaxError: invalid syntax (<ipython-input-355-723a60b81a3e>, line 72)

In [356]:
class Node(ABC):
    """
    A representation of a single board state.
    MCTS works by constructing a tree of these Nodes.
    Could be e.g. a chess or checkers board state.
    """

    @abstractmethod
    def find_children(self):
        "All possible successors of this board state"
        return set()

    @abstractmethod
    def find_random_child(self):
        "Random successor of this board state (for more efficient simulation)"
        return None

    @abstractmethod
    def is_terminal(self):
        "Returns True if the node has no children"
        return True

In [357]:
#generate random graph of preset size
def generate_graph(vertices):
    return nx.generators.random_graphs.gnp_random_graph(vertices,
                np.random.uniform(0,0.5))

In [358]:
class VertexCoverInstance(Node):
    def __init__(self, graph, cover = []):
        self.graph = graph
        self.cover = cover
        
    def find_children(self):
        possiblemoves = []
        if self.is_terminal():  # If the game is finished then no moves can be made
            return set(possiblemoves)
        for i in list(self.graph.nodes):
            H = self.graph.copy()
            neigh = H.neighbors(i)
            H.remove_node(i)
            H.add_node(i)
            possiblemoves.append(VertexCoverInstance(H, self.cover+[i]))
        return possiblemoves

    def find_random_child(self):
        if self.is_terminal():
            return None  # If the game is finished then no moves can be made
        temp = self.find_children()
        return random.sample(temp,1)[0]

    def is_terminal(self):
        return nx.classes.function.is_empty(self.graph)

    def to_pretty_string(self):
        return str(list(self.graph.nodes()))
    
    def get_cover(self):
        return self.cover
    
    def get_reward(self):
        if self.is_terminal():
             return 0
        return -1
    

In [359]:
def play_game(G, net, num_rollouts=80, display = True):
    tree = MCTS(net)
    board = VertexCoverInstance(G)
    while True:
        if board.is_terminal():
            break
        for _ in range(num_rollouts):
            tree.do_rollout(board)
        board = tree.choose(board)
            
    vc = board.get_cover()
    if display:
        print("\n\nMCTS APPROX\n")
        print(vc)
        color_map = []
        for node in G:
            if node in vc:
                color_map.append('red')
            else: 
                color_map.append('blue')  
        plt.figure(1)
        nx.draw(G, node_color=color_map, with_labels=True)
        
    return vc

In [360]:
def self_play(G, net, num_rollouts=80):
    tree = MCTS(net)
    board = VertexCoverInstance(G)
    data = []
    while True:
        if board.is_terminal():
            break
        for _ in range(num_rollouts):
            tree.do_rollout(board)
        new_board, policy = tree.choose_by_policy(board)
        record = from_networkx(board.graph)
        record.y = torch.tensor(policy).reshape(1,-1)
        data.append(record)
        board = new_board 
        
    return data

In [361]:
from networkx.algorithms import approximation as appx
import itertools

def show_optimal_vc(G, display=True):
    def findsubsets(s): 
        lists =[list(itertools.combinations(s, n)) for n in range(len(s))]
        return list(itertools.chain.from_iterable(lists))
    
    powerset = findsubsets(list(G.nodes()))    
    for s in powerset:
        H = G.copy()
        H.remove_nodes_from(s)
        if nx.classes.function.is_empty(H):
            opt = list(s)
            break
    if display:
        print("\n\nOPTIMAL\n")
        print(opt)
        color_map = []
        for node in G:
            if node in opt:
                color_map.append('orange')
            else: 
                color_map.append('green')
        plt.figure(2)
        nx.draw(G, node_color=color_map, with_labels=True)
        
    return opt

In [362]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_networkx

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 16)
        self.conv3 = GCNConv(16, 16)
        self.conv4 = GCNConv(16, 16)
        self.conv_prob = GCNConv(16, 1)

    # expects a torch_geometric Data object
    def forward(self, data):
        num_nodes = data.num_nodes
        x = torch.tensor([[1.0] for _ in range(num_nodes)])
        edge_index = data.edge_index
        x = self.conv1(x ,edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        edge_index = data.edge_index
        x = self.conv2(x ,edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv3(x ,edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv4(x ,edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        probs = self.conv_prob(x, edge_index).reshape(-1,num_nodes)
        return F.softmax(probs,dim=1)
    
    def predict(self, instance):
        data = from_networkx(instance.graph)
        return self.forward(data).flatten().tolist()
        
net = Net()

In [ ]:
from torch_geometric.data import DataLoader

TRAIN_GRAPH_SIZE = 10
TEST_GRAPH_SIZE = 10

NUM_ITERS = 20
NUM_SELF_GAMES = 50
NUM_EPOCHS = 100
NUM_EVAL_SIMS = 25

scores = []

net = Net()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)
criterion=torch.nn.BCELoss()

for i in range(NUM_ITERS):
    print('Iteration %d' % (i+1))
    
    # get self-play records
    print("Getting self-play records")
    records = []
    for i in range(NUM_SELF_GAMES):
        A = generate_graph(TRAIN_GRAPH_SIZE)
        records.extend(self_play(A, net))
        if i % 10 == 9:
            print("  Generated game %d" % (i+1))
    
    # training
    print("Training")
    data_loader = DataLoader(records, batch_size=16, shuffle=True)
    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        num_batches = 0
        for batch in data_loader:

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(batch)
            loss = criterion(outputs, batch.y)
            total_loss += loss.item()
            num_batches += 1
            loss.backward()
            optimizer.step()

        # print statistics
        if epoch % 10 == 9:
            print('  Epoch %d loss: %.3f' %
                  (epoch+1, total_loss / num_batches))
    
    # evaluate on larger test graphs
    print("Evaluating")
    count_success = 0
    for x in range(NUM_EVAL_SIMS):
        A = generate_graph(TEST_GRAPH_SIZE)
        vc = play_game(A, net, display=False)
        opt = show_optimal_vc(A, display=False)
        if len(vc)<=len(opt)*1.1:
            count_success=count_success+1
    print('Score: %.3f' % (count_success / NUM_EVAL_SIMS))
    scores.append(count_success / NUM_EVAL_SIMS)
    

plt.plot(scores)
plt.ylabel('Performance')
plt.xlabel('Self Play / Training Iterations')
plt.title('Percentage of Good AlphaMinVertex Approximations during Training')
plt.show()

Iteration 1
Getting self-play records
  Generated game 10
  Generated game 20
  Generated game 30
  Generated game 40
  Generated game 50
Training
Epoch 10 loss: 0.508
Epoch 20 loss: 0.503
Epoch 30 loss: 0.499
Epoch 40 loss: 0.499
Epoch 50 loss: 0.498
Epoch 60 loss: 0.496
Epoch 70 loss: 0.496
Epoch 80 loss: 0.494
Epoch 90 loss: 0.497
Epoch 100 loss: 0.496
Evaluating
Score: 0.640
Iteration 2
Getting self-play records
  Generated game 10
  Generated game 20
  Generated game 30
  Generated game 40
  Generated game 50
Training
Epoch 10 loss: 0.494
Epoch 20 loss: 0.489
Epoch 30 loss: 0.492
Epoch 40 loss: 0.489
Epoch 50 loss: 0.491
Epoch 60 loss: 0.490
Epoch 70 loss: 0.491
Epoch 80 loss: 0.495
Epoch 90 loss: 0.490
Epoch 100 loss: 0.491
Evaluating
Score: 0.600
Iteration 3
Getting self-play records
  Generated game 10
  Generated game 20
  Generated game 30
  Generated game 40
  Generated game 50
Training
Epoch 10 loss: 0.489
Epoch 20 loss: 0.488
Epoch 30 loss: 0.486
Epoch 40 loss: 0.486
Epoch

/usr/local/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([11, 10])) that is different to the input size (torch.Size([1, 110])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch 10 loss: 0.489
Epoch 20 loss: 0.489
Epoch 30 loss: 0.489
Epoch 40 loss: 0.489
Epoch 50 loss: 0.490
Epoch 60 loss: 0.489
Epoch 70 loss: 0.489
Epoch 80 loss: 0.489
Epoch 90 loss: 0.493
Epoch 100 loss: 0.493
Evaluating
Score: 0.360
Iteration 6
Getting self-play records
  Generated game 10
  Generated game 20
  Generated game 30
  Generated game 40
  Generated game 50
Training
Epoch 10 loss: 0.488
Epoch 20 loss: 0.492
Epoch 30 loss: 0.491
Epoch 40 loss: 0.490
Epoch 50 loss: 0.492
Epoch 60 loss: 0.491
Epoch 70 loss: 0.489
Epoch 80 loss: 0.487
Epoch 90 loss: 0.492
Epoch 100 loss: 0.490
Evaluating
Score: 0.560
Iteration 7
Getting self-play records
  Generated game 10
  Generated game 20
  Generated game 30
  Generated game 40
  Generated game 50
Training


/usr/local/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([4, 10])) that is different to the input size (torch.Size([1, 40])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch 10 loss: 0.482
Epoch 20 loss: 0.486
Epoch 30 loss: 0.483
Epoch 40 loss: 0.479
Epoch 50 loss: 0.480
Epoch 60 loss: 0.484
Epoch 70 loss: 0.486
Epoch 80 loss: 0.481
Epoch 90 loss: 0.483
Epoch 100 loss: 0.483
Evaluating
Score: 0.680
Iteration 8
Getting self-play records
  Generated game 10
  Generated game 20
  Generated game 30
  Generated game 40
  Generated game 50
Training
Epoch 10 loss: 0.489
Epoch 20 loss: 0.486
Epoch 30 loss: 0.487
